In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('../data/processed/0.1-initial-exploration.csv',index_col=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.loc[:,'positionOrder'] = pd.to_numeric(df.loc[:, 'positionOrder'], errors='coerce')
df.loc[:,'ageAtRace'] = pd.to_numeric(df.loc[:, 'ageAtRace'], errors='coerce')
df.loc[:,'ageAtDebut'] = pd.to_numeric(df.loc[:, 'ageAtDebut'], errors='coerce')
df.loc[:,'yearsExperience'] = pd.to_numeric(df.loc[:, 'yearsExperience'], errors='coerce')

**Sort by year and round**

In [13]:
df = df.sort_values(by=['year','round'])

**Carry forward last Driver Standings statistics**

In [14]:
df['driverStandingsPoints'] = df.groupby(['driverId'])['driverStandingsPoints'].transform(lambda x: x.ffill())
df['driverStandingsPosition'] = df.groupby(['driverId'])['driverStandingsPosition'].transform(lambda x: x.ffill())
df['driverStandingsWins'] = df.groupby(['driverId'])['driverStandingsWins'].transform(lambda x: x.ffill())

**Carry forward last Constructor Standings statistics**

In [15]:
df['constructorStandingsPoints'] = df.groupby(['driverId'])['constructorStandingsPoints'].transform(lambda x: x.ffill())
df['constructorStandingsPosition'] = df.groupby(['driverId'])['constructorStandingsPosition'].transform(lambda x: x.ffill())
df['constructorStandingsWins'] = df.groupby(['driverId'])['constructorStandingsWins'].transform(lambda x: x.ffill())

**Drop useless columns**

In [16]:
# df = df.drop(['resultId','raceId','constructorId','position','positionText','pointsGained',
# 'laps','time_x','milliseconds','fastestLapSpeed','fastestLapTime','fastestLap','statusId'
# ,'circuitName','date','dateOfDebut','status','driverStandingsId'], 1)

**Columns with null values**

In [17]:
print('Columns with null values:\n', df.columns[df.isna().any()].tolist())

Columns with null values:
 ['driverStandingsId', 'lastRaceRank', 'constructorStandingsId', 'constructorStandingsPoints', 'constructorStandingsPosition', 'constructorStandingsWins', 'previousRaceGridStart', 'previousRacePosition', 'racesWon', 'racesRetired', 'racesFinished', 'polePositions', 'racesWonByConstructor', 'racesRetiredByConstructor', 'percentageOfBestQuali']


**Replace Nans with 0 or -1 depending on the column**

In [18]:
df[['racesWon','racesRetired','racesFinished','polePositions','racesWonByConstructor','racesRetiredByConstructor']] = df[['racesWon','racesRetired','racesFinished','polePositions','racesWonByConstructor','racesRetiredByConstructor']].fillna(value=0)
df[['lastRaceRank','previousRaceGridStart','previousRacePosition']] = df[['lastRaceRank','previousRaceGridStart','previousRacePosition']].fillna(value=-1)

**Chech which columns still have missing values**

In [19]:
print('Columns with null values:\n', df.columns[df.isna().any()].tolist())

Columns with null values:
 ['driverStandingsId', 'constructorStandingsId', 'constructorStandingsPoints', 'constructorStandingsPosition', 'constructorStandingsWins', 'percentageOfBestQuali']


**(Should be mean of the current year up until that point only!!!)**

In [20]:
#df['percentageOfBestQuali'] = df.groupby(['driverId','year'])['percentageOfBestQuali'].transform(lambda x: x.fillna(x.mean()))
df['percentageOfBestQuali'] = df.groupby(['driverId','year'])['percentageOfBestQuali'].transform(lambda x: x.ffill())